In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

In [60]:
import numpy as np

class ModelPipeline():
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("noobiebuilder/bert-fintuned-classification")
        
        labels = ['normal', 'offensive', 'hatespeech']
        id2label = {i: label for i, label in enumerate(labels)}
        label2id = {v: k for k, v in id2label.items()}
        
        self.model = AutoModelForSequenceClassification.from_pretrained("noobiebuilder/bert-fintuned-classification", 
                                                                        id2label=id2label, label2id=label2id,)

    def predict_proba(self, inputs):
        sentence = inputs
        sentence_token = sentence.split()
        sentence_token = self.tokenizer.encode(sentence_token, is_split_into_words=True)
        model_input = torch.tensor(sentence_token).unsqueeze(0)

        self.model.eval()
        with torch.no_grad():
            outputs = self.model(model_input, output_attentions=False)
            
        probas = F.softmax(outputs.logits, dim = 1).detach().numpy()
        return probas

In [61]:
pipe = ModelPipeline()

In [62]:
data = "Fuck this bitch"

In [63]:
pipe.predict_proba(data)

array([[0.12527812, 0.8312805 , 0.04344134]], dtype=float32)

In [49]:
import modelbit
mb = modelbit.login()

In [64]:
def predict_hate(sentence):
    #pipe = ModelPipeline()
    prediction = pipe.predict_proba(sentence)
    return prediction


In [65]:
predict_hate(data)

array([[0.12527812, 0.8312805 , 0.04344134]], dtype=float32)

In [66]:
mb.deploy(predict_hate)

Encrypting 'pipe': 450MB [00:02, 173MB/s]                            
Uploading 'pipe': 100%|██████████| 402M/402M [00:09<00:00, 43.0MB/s] 


In [67]:
import json
import requests

requests.post(
    "https://usc.ap-south-1.modelbit.com/v1/predict_hate/latest",
    headers={"Content-Type": "application/json"},
    data=json.dumps(
        {
            "data": [
                "Fuck this bitch",
            ]
        }
    ),
).json()

{'data': [[0.12527811527252197, 0.831280529499054, 0.04344134405255318]]}

In [13]:
import requests
import json
import numpy as np

def get_prediction_lime(sentences):
    batch = []
    for index, sentence in enumerate(sentences):
        batch_element = [int(index)]
        batch_element.append(sentence)
        batch.append(batch_element)
    
    predictions = requests.post("https://usc.ap-south-1.modelbit.com/v1/predict_hate/latest",
                                headers={"Content-Type": "application/json"},
                                data=json.dumps({"data": batch}),).json()
    

    #print(prediction)
    removed_index_pred_batch = []
    prediction_batch = predictions['data']
    for pred in prediction_batch:
        removed_index_pred_batch.append(np.array(pred[1][0]))

    return np.array(removed_index_pred_batch)

In [14]:
get_prediction_lime(["Fuck this bitch", "Fuck this", "Fuck"])

array([[0.12527812, 0.83128053, 0.04344134],
       [0.43427444, 0.50839931, 0.05732629],
       [0.41864759, 0.51479369, 0.06655867]])

In [15]:
from lime.lime_text import LimeTextExplainer

explainer = LimeTextExplainer(class_names=['normal', 'offensive', 'hatespeech'])
str_to_predict = "This bitch always effected me"
exp = explainer.explain_instance(str_to_predict, get_prediction_lime)

In [92]:
prediction = [[0, [[0.12527811527252197, 0.831280529499054, 0.04344134405255318]]],
 [1, [[0.43427443504333496, 0.5083993077278137, 0.0573262944817543]]],
 [2, [[0.4186475872993469, 0.5147936940193176, 0.06655866652727127]]]]

In [93]:
prediction[0][1][0]

[0.12527811527252197, 0.831280529499054, 0.04344134405255318]

In [94]:
for pred in prediction:
    print(pred[1][0])

[0.12527811527252197, 0.831280529499054, 0.04344134405255318]
[0.43427443504333496, 0.5083993077278137, 0.0573262944817543]
[0.4186475872993469, 0.5147936940193176, 0.06655866652727127]


In [ ]:
exp.show_in_notebook(text=str_to_predict)